In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity



# Cargar el dataset
data = pd.read_csv('products.csv')


In [4]:
# Crear una tabla de usuario-producto (poniendo 1 donde el usuario ha comprado el producto)
user_product_matrix = pd.pivot_table(data, index='usuario_id', columns='nombre_producto', aggfunc='size', fill_value=0)

# Calcular la matriz de similitud de coseno entre usuarios
user_similarity_matrix = cosine_similarity(user_product_matrix)

# Convertir la matriz de similitud en un DataFrame
user_similarity_df = pd.DataFrame(user_similarity_matrix, index=user_product_matrix.index, columns=user_product_matrix.index)

In [5]:
def get_user_recommendations(user_id, train_data, user_similarity_df, num_recommendations=5):
    # Verificar si el usuario está en los datos de entrenamiento
    if user_id not in train_data.index:
        print(f"El usuario {user_id} no tiene historial de compras.")
        top_products = train_data.sum().nlargest(num_recommendations).index
        return list(top_products)

    # Obtener la fila de similitud del usuario
    user_similarity_row = user_similarity_df.loc[user_id]

    # Obtener usuarios similares ordenados por similitud, excluyendo al propio usuario
    similar_users = user_similarity_row.drop(user_id).sort_values(ascending=False)

    # Generar las recomendaciones basadas en los usuarios similares
    recommendations = []
    user_purchased_products = set(train_data.loc[user_id][train_data.loc[user_id] > 0].index)

    for similar_user_id in similar_users.index:
        similar_user_history = train_data.loc[similar_user_id]
        similar_user_purchased_products = set(similar_user_history[similar_user_history > 0].index)

        # Recomendar productos que el usuario similar ha comprado pero el usuario actual no
        recommended_products = similar_user_purchased_products - user_purchased_products
        recommendations.extend(recommended_products)

        # Limitar el número de recomendaciones
        if len(recommendations) >= num_recommendations:
            break

    # Eliminar duplicados y limitar el número de recomendaciones
    recommendations = list(set(recommendations))[:num_recommendations]

    return recommendations


In [7]:
# Ejemplo de uso con un usuario existente
user_id = 'usuario2'
recommended_products = get_user_recommendations(user_id, user_product_matrix, user_similarity_df, num_recommendations=5)
print(f"Recommended products for {user_id}: {recommended_products}")


Recommended products for usuario2: ['Trident Sabor Menta', 'Fritos Chorizo y Limón', 'Gansito Marinela', 'Triki Trakes Gamesa', 'Sprite 600ml']
